In [ ]:
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
hour_difference = 2.5

In [ ]:
f = open("data.txt", "r")
txt_data = f.read()
print(txt_data)

In [ ]:
def stringtodate(string):
    datetime_object = datetime.strptime(string, '%d/%m')
    datetime_object = datetime_object.replace(year = datetime.now().year).date()
    return datetime_object

In [ ]:
def stringtotime(time, date):
    date = stringtodate(date)
    time = datetime.strptime(time, '%H:%M').time()
    date_time = datetime.combine(date, time)
    return date_time

In [ ]:
def days_hours_minutes(td):
    return td.total_seconds() / 60

In [ ]:
data = pd.DataFrame(columns = ["Date","Start Time", "End Time", "Comment"])

with open("data.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        if '/' in line:
            date = line.rstrip('\n')
        elif line == '\n':
            continue
        else:
            start_time, end_time = line.split('-')
            start_time, end_time = start_time.strip(), end_time.strip()
            
            if '(' in end_time:
                end_time, comment = end_time.split('(')
                end_time = end_time.strip()
                comment = comment.strip('()')
            else:
                end_time, comment = end_time, None

            row = {"Date":date, "Start Time":stringtotime(start_time, date), "End Time":stringtotime(end_time, date), "Comment":comment}
            data = data.append(row, ignore_index = True)

data['Date'] = data['Date'].apply(stringtodate)
data = data.sort_values(['Date'])
data['Total Time'] = data['End Time'] - data['Start Time']
data['Total Minutes'] = data['Total Time'].apply(days_hours_minutes)
data['Client Start Time'] = data['Start Time'] + pd.to_timedelta(hour_difference, unit='h')
data['Client End Time'] = data['End Time'] + pd.to_timedelta(hour_difference, unit='h')

data = data[data.Comment != 'break']

data['Start Time'] = data['Start Time'].dt.strftime('%H:%M')
data['End Time'] = data['End Time'].dt.strftime('%H:%M')
data['Client Start Time'] = data['Client Start Time'].dt.strftime('%H:%M')
data['Client End Time'] = data['Client End Time'].dt.strftime('%H:%M')

In [ ]:
total_minutes_by_date = data.groupby(by='Date').sum()
total_minutes_by_date

In [ ]:
data

In [ ]:
data['Total Time'].sum()